In [1]:
import pandas as pd
import numpy as np
import itertools
import re
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import os
import openpyxl
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# Download edilen takım stats excelleri tek bir klasörde toplanmış olmalı
# dizin kontrol edilmeli
# for döngüsüne girecek liste belirlenmeli
lig=[]
ligs=[]
    
dizin="C:/Users/umit/data science/KENDİ ÇALIŞMALARIM/- PREMIER LEAGUE/- TAKIMLAR"
premier=os.listdir(dizin)
for b in premier:
    ligs.append(b)
    a=b[11:-5]
    lig.append(a)


In [3]:
def FileProcessing(lig):
    
    try:
        os.remove("DF.csv")
        pass
    except:
        pass
        
    for l in lig:
        # veri setinin import edilmesi, 
        # eksik verilerin atılması (ilk 2 satır değilse kontrol edilmeli)
        # yeniden indexleme
        df = pd.read_excel("C:/Users/umit/data science/KENDİ ÇALIŞMALARIM/- PREMIER LEAGUE/- TAKIMLAR/"+"Team Stats "+str(l)+".xlsx")
        df = df.dropna()
        df=df.rename(index={j: i for i, j in enumerate(df.index)})

        # Karşılaşma takımlarının ayrımı, atılan goller ve yenilen gollerin belirlenmesi -- regepx-- uzunluklar kontrol edildi.
        EvSahibi      = []
        Deplasman     = []
        EvSahibiSkor  = []
        DeplasmanSkor = []
        Sema          = []
        db = [i for i in df['Grubun veritabanı']]
        db = set(db)
        
        for index in df['Mac']:
            nesne1 = re.match(".+-",index)    # - öncesi bütün karakterler
            nesne1 = nesne1.group()
            nesne1 = nesne1[:-2]

            if nesne1 == 'Brighton & Hove Albion':
                    nesne1 = 'Brighton'
                    EvSahibi.append(nesne1)

            elif nesne1 == 'Olympiakos Piraeus':
                nesne1 = 'Olympiacos Piraeus'
                EvSahibi.append(nesne1)

            else:
                EvSahibi.append(nesne1)
                #print('Dikkat: {} Evsahibi takımı isim olarak yanlış yazılışmış olabilir'.format(nesne1))

            nesne2 = re.search('-.+([A-Z]|\')\w+',index) # - sonrası bütün karakterler boşluğa kadar
            print(nesne2)
            nesne2 = nesne2.group()
            nesne2 = nesne2[2:]
            r = re.compile(".*{}".format(nesne2))
            newlist = list(filter(r.match, db))
            search = ''.join(newlist)

            if search:
                Deplasman.append(search)

            else:
                if nesne2 == 'Brighton & Hove Albion':
                    nesne2 = 'Brighton'
                    Deplasman.append(nesne2)

                elif nesne2 == 'Olympiakos Piraeus':
                    nesne2 = 'Olympiacos Piraeus'
                    Deplasman.append(nesne2)

                else:
                    Deplasman.append(nesne2)
                    print('Dikkat: {} Deplasman takımı isim olarak yanlış yazılışmış olabilir'.format(nesne2))

            nesne3 = re.search(".+:",index)       # : öncesi bütün karakterler --->/ alternatif ]-.+|[0-9]
            nesne3 = nesne3.group()
            nesne3 = nesne3[len(nesne3)-2:-1]
            EvSahibiSkor.append(int(nesne3))

            nesne4 = re.search(":[0-9]+",index)   # : sonrası bütün karakterler
            nesne4 = nesne4.group()
            nesne4 = nesne4[1:]
            DeplasmanSkor.append(int(nesne4))

        for i in df['Şema']:
            nesne = re.search("(?<=\\().*?(?=\\))", i)
            nesne = nesne.group()
            nesne  = i.replace(nesne, '').replace('()','').replace(' ','')
            Sema.append(nesne)

        # yeni eklenen kolonlar ve Tarih sütunu
        EvSahibi=pd.DataFrame(data=EvSahibi,index=range(len(EvSahibi)),columns=["EvSahibi"])
        EvSahibiSkor=pd.DataFrame(data=EvSahibiSkor,index=range(len(EvSahibiSkor)),columns=["EvSahibiSkor"])
        Deplasman=pd.DataFrame(data=Deplasman,index=range(len(Deplasman)),columns=["Deplasman"])
        DeplasmanSkor=pd.DataFrame(data=DeplasmanSkor,index=range(len(DeplasmanSkor)),columns=["DeplasmanSkor"])
        Sema=pd.DataFrame(data=Sema,index=range(len(Sema)),columns=["Sema"])
        df['Şema'] = Sema['Sema']
        df=df.drop(["Süre","Mac"],axis=1)
        df=pd.concat([EvSahibi, EvSahibiSkor, Deplasman, DeplasmanSkor, df],axis=1)
        df['Tarih'] = df['Tarih'].apply(lambda x: pd.to_datetime(x))

        # kolonlar yeniden adlandırıldı
        df=df.rename(columns={
            'EvSahibi':'EvSahibi',
            'EvSahibiSkor':'EvSahibiSkor',
            'Deplasman':'Deplasman',
            'DeplasmanSkor':'DeplasmanSkor',
            'Tarih':'Tarih',
            'Yarışma':'Yarisma',
            'Grubun veritabanı':'Grubun_veritabani',
            'Şema':'Şema',
            'Goller':'Atilan_Gol',
            'xG':'xG',
            'Şutlar / Hedefe':'Sutlar',
            'Unnamed: 9':'Sutlar_hedef',
            'Unnamed: 10':'Sut_yuzdesi',
            'Paslar / doğru':'Paslar',
            'Unnamed: 12':'Paslar_dogru',
            'Unnamed: 13':'Pas_yuzdesi',
            'Top hakimiyeti, %':'Top_hakimiyeti_yuzdesi',
            'Kayıplar / Düşük / Orta / Yüksek':'Kayiplar',
            'Unnamed: 16':'Kayiplar_Dusuk',
            'Unnamed: 17':'Kayiplar_Orta',
            'Unnamed: 18':'Kayiplar_Yuksek',
            'Kurtarışlar / Düşük / Orta / Yüksek':'Kurtarislar',
            'Unnamed: 20':'Kurtarislar_Dusuk',
            'Unnamed: 21':'Kurtarislar_Orta',
            'Unnamed: 22':'Kurtarislar_Yuksek',
            'Çekişmeler / kazanilan':'Cekismeler',
            'Unnamed: 24':'Cekismeler_kazanilan',
            'Unnamed: 25':'Cekismeler_yuzdesi'
        })

        # Saha, Rakip ve Yenilen Gollerin kolon haline getirilerek daha belirginleştirilmesi
        evsahibi=df["EvSahibi"]
        DBKlup=df["Grubun_veritabani"]
        deplasman=df["Deplasman"]
        EvSahibiSkor=df['EvSahibiSkor']
        DeplasmanSkor=df['DeplasmanSkor']
        Saha  = []
        Rakip = []
        Yenilen_Gol = []
        for ev, kl, dep, evskr, depskr in zip(evsahibi, DBKlup, deplasman, EvSahibiSkor, DeplasmanSkor):
            if ev==kl:
                Saha.append("Evsahibi")
                Rakip.append(dep)
                Yenilen_Gol.append(depskr)
            else:
                Saha.append("Deplasman")
                Rakip.append(ev)
                Yenilen_Gol.append(evskr)
        Saha=pd.DataFrame(data=Saha,index=range(len(Saha)),columns=["Saha"])
        Rakip=pd.DataFrame(data=Rakip,index=range(len(Rakip)),columns=["Rakip"])
        Yenilen_Gol=pd.DataFrame(data=Yenilen_Gol,index=range(len(Yenilen_Gol)),columns=["Yenilen_Gol"])
        df=pd.concat([Saha,Rakip, Yenilen_Gol, df],axis=1)
        #roden=roden.dropna()

        # tablonun daha anlamlı hale gelmesi için kolonların yeniden sıralanması
        bir_kisim   = df['Grubun_veritabani']
        iki_kisim   = df['Tarih']
        uc_kisim    = df['Yarisma']
        dort_kisim  = df['Saha']
        bes_kisim   = df['Şema']
        alti_kisim  = df['Rakip']
        Son_kisim   = df.loc[:,"xG":]
        yedi_kisim  = df['Atilan_Gol']
        sekiz_kisim = df['Yenilen_Gol']
        df=pd.concat([bir_kisim, iki_kisim, uc_kisim, dort_kisim, bes_kisim, alti_kisim, Son_kisim, sekiz_kisim, yedi_kisim],axis=1)

        # kolon tiplerinin düzenlemesi
        df['Atilan_Gol'] = df['Atilan_Gol'].astype('int')
        df['Yenilen_Gol'] = df['Yenilen_Gol'].astype('int')
        df['Sutlar'] = df['Sutlar'].astype('int')
        df['Sutlar_hedef'] = df['Sutlar_hedef'].astype('int')
        df['Paslar'] = df['Paslar'].astype('int')
        df['Paslar_dogru'] = df['Paslar_dogru'].astype('int')
        df['Kayiplar'] = df['Kayiplar'].astype('int')
        df['Kayiplar_Dusuk'] = df['Kayiplar_Dusuk'].astype('int')
        df['Kayiplar_Orta'] = df['Kayiplar_Orta'].astype('int')
        df['Kayiplar_Yuksek'] = df['Kayiplar_Yuksek'].astype('int')
        df['Kurtarislar'] = df['Kurtarislar'].astype('int')
        df['Kurtarislar_Dusuk'] = df['Kurtarislar_Dusuk'].astype('int')
        df['Kurtarislar_Orta'] = df['Kurtarislar_Orta'].astype('int')
        df['Kurtarislar_Yuksek'] = df['Kurtarislar_Yuksek'].astype('int')
        df['Cekismeler'] = df['Cekismeler'].astype('int')
        df['Cekismeler_kazanilan'] = df['Cekismeler_kazanilan'].astype('int')
        
        # verilerin tabloya açık bir şekilde işlenmesi
        df.to_csv("DF.csv", index=False, mode = 'a', header=False)
    print(df.columns)

In [4]:
FileProcessing(lig)

<re.Match object; span=(9, 18), match='- Arsenal'>
<re.Match object; span=(9, 18), match='- Arsenal'>
<re.Match object; span=(8, 27), match='- Tottenham Hotspur'>
<re.Match object; span=(8, 27), match='- Tottenham Hotspur'>
<re.Match object; span=(8, 23), match='- AFC Wimbledon'>
<re.Match object; span=(8, 23), match='- AFC Wimbledon'>
<re.Match object; span=(8, 17), match='- Arsenal'>
<re.Match object; span=(8, 17), match='- Arsenal'>
<re.Match object; span=(8, 22), match='- Norwich City'>
<re.Match object; span=(8, 22), match='- Norwich City'>
<re.Match object; span=(16, 25), match='- Arsenal'>
<re.Match object; span=(16, 25), match='- Arsenal'>
<re.Match object; span=(21, 30), match='- Arsenal'>
<re.Match object; span=(21, 30), match='- Arsenal'>
<re.Match object; span=(8, 17), match='- Chelsea'>
<re.Match object; span=(8, 17), match='- Chelsea'>
<re.Match object; span=(10, 19), match='- Arsenal'>
<re.Match object; span=(10, 19), match='- Arsenal'>
<re.Match object; span=(18, 27), m

### Kolonların Adlandırılması

In [5]:
coll = ['Grubun_veritabani', 'Tarih', 'Yarisma', 'Saha', 'Şema', 'Rakip', 'xG',
       'Sutlar', 'Sutlar_hedef', 'Sut_yuzdesi', 'Paslar', 'Paslar_dogru',
       'Pas_yuzdesi', 'Top_hakimiyeti_yuzdesi', 'Kayiplar', 'Kayiplar_Dusuk',
       'Kayiplar_Orta', 'Kayiplar_Yuksek', 'Kurtarislar', 'Kurtarislar_Dusuk',
       'Kurtarislar_Orta', 'Kurtarislar_Yuksek', 'Cekismeler',
       'Cekismeler_kazanilan', 'Cekismeler_yuzdesi', 'Yenilen_Gol',
       'Atilan_Gol']
df=pd.read_csv("DF.csv", names=coll)
df=df.rename(index={j: i for i, j in enumerate(df.index)})
df['Tarih'] = df['Tarih'].apply(lambda x: pd.to_datetime(x))
df=df.drop_duplicates(keep='last')
df=df.rename(index={j: i for i, j in enumerate(df.index)})
# df.to_excel("df1.xlsx",index=False)

In [100]:
#df

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1492 entries, 0 to 1491
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Grubun_veritabani       1492 non-null   object        
 1   Tarih                   1492 non-null   datetime64[ns]
 2   Yarisma                 1492 non-null   object        
 3   Saha                    1492 non-null   object        
 4   Şema                    1492 non-null   object        
 5   Rakip                   1492 non-null   object        
 6   xG                      1492 non-null   float64       
 7   Sutlar                  1492 non-null   int64         
 8   Sutlar_hedef            1492 non-null   int64         
 9   Sut_yuzdesi             1492 non-null   float64       
 10  Paslar                  1492 non-null   int64         
 11  Paslar_dogru            1492 non-null   int64         
 12  Pas_yuzdesi             1492 non-null   float64      

GRAFİK ÇALIŞMALARI

Index(['Grubun_veritabani', 'Tarih', 'Yarisma', 'Saha', 'Şema', 'Rakip', 'xG',
       'Sutlar', 'Sutlar_hedef', 'Sut_yuzdesi', 'Paslar', 'Paslar_dogru',
       'Pas_yuzdesi', 'Top_hakimiyeti_yuzdesi', 'Kayiplar', 'Kayiplar_Dusuk',
       'Kayiplar_Orta', 'Kayiplar_Yuksek', 'Kurtarislar', 'Kurtarislar_Dusuk',
       'Kurtarislar_Orta', 'Kurtarislar_Yuksek', 'Cekismeler',
       'Cekismeler_kazanilan', 'Cekismeler_yuzdesi', 'Yenilen_Gol',
       'Atilan_Gol'],
      dtype='object')

Slice = df.loc[df.Grubun_veritabani=="Arsenal",["Sutlar_hedef","Atilan_Gol"]]

kolon1 = 'Sutlar_hedef'
kolon2 = 'Atilan_Gol'
x      = df[kolon1]
y      = df[kolon2]

plt.scatter(x,y)
plt.xlabel(kolon1)
plt.ylabel(kolon2)

# MODELLEME

### Labelencoder (Takımları unique değer olarak kolonlama)

In [102]:
setTakimlar=list(set(df['Grubun_veritabani']))
Setdf = pd.DataFrame(columns=setTakimlar)
zeros = list(np.zeros(len(df['Grubun_veritabani']), dtype=int))
sözlükTakimlar = {i: zeros for i in setTakimlar}
dfTakimlar = pd.DataFrame(sözlükTakimlar)
for i, tk, rkp in zip(range(len(df['Grubun_veritabani'])), df['Grubun_veritabani'], df['Rakip']):
    dfTakimlar.loc[i,tk] = 1
    dfTakimlar.loc[i,rkp] = 1
df = pd.concat([dfTakimlar, df],axis=1)
df.to_excel("df.xlsx",index=False)
df=df.drop(["Grubun_veritabani", "Rakip", "Tarih"],axis=1)

In [103]:
lb=LabelEncoder()

lb.fit_transform(df["Yarisma"]) #2
Yarisma=pd.get_dummies(df.iloc[:,122:123],columns=["Yarisma"], dtype=int)

lb.fit_transform(df["Saha"]) #3
Saha=pd.get_dummies(df.iloc[:,123:124],columns=["Saha"], dtype=int)
Saha.drop(['Saha_Deplasman'],axis=1,inplace=True)
Saha=Saha.rename(columns={'Saha_Evsahibi':'Saha'})

lb.fit_transform(df["Şema"]) #4
Sema=pd.get_dummies(df.iloc[:,124:125],columns=["Şema"], dtype=int)

birinci_kisim=df.iloc[:,:122]
ikinci_kisim=df.iloc[:,125:-2]

X=pd.concat([Sema, Yarisma, Saha, birinci_kisim, ikinci_kisim],axis=1) #

### Ordinal Encoder

ordi=OrdinalEncoder()

Yarisma = ordi.fit_transform(df[["Yarisma"]]) #2
Yarisma=pd.DataFrame(data=Yarisma,index=range(len(Yarisma)),columns=["Yarisma"])

Saha = ordi.fit_transform(df[["Saha"]]) #3
Saha=pd.DataFrame(data=Saha,index=range(len(Saha)),columns=["Saha"])

Sema = ordi.fit_transform(df[["Şema"]]) #4
Sema=pd.DataFrame(data=Sema,index=range(len(Sema)),columns=["Sema"])

birinci_kisim=df.iloc[:,:122]
ikinci_kisim=df.iloc[:,125:-2]

X=pd.concat([Sema, Yarisma, Saha, birinci_kisim, ikinci_kisim],axis=1) #

### Labelencoder 2 (Takımları ilkel yöntemle kolonlama)

df=df.drop(["Tarih"],axis=1)
lb=LabelEncoder()

lb.fit_transform(df["Grubun_veritabani"]) #1
Grubun_veritabani=pd.get_dummies(df.iloc[:,0:1],columns=["Grubun_veritabani"], dtype=int)

lb.fit_transform(df["Yarisma"]) #2
Yarisma=pd.get_dummies(df.iloc[:,1:2],columns=["Yarisma"], dtype=int)

lb.fit_transform(df["Saha"]) #3
Saha=pd.get_dummies(df.iloc[:,2:3],columns=["Saha"], dtype=int)
Saha.drop(['Saha_Deplasman'],axis=1,inplace=True)
Saha=Saha.rename(columns={'Saha_Evsahibi':'Saha'})

lb.fit_transform(df["Şema"]) #4
Sema=pd.get_dummies(df.iloc[:,3:4],columns=["Şema"], dtype=int)

lb.fit_transform(df["Rakip"]) #5
Rakip=pd.get_dummies(df.iloc[:,4:5],columns=["Rakip"], dtype=int)

ikinci_kisim=df.iloc[:,5:24]

X=pd.concat([Yarisma,Grubun_veritabani,Rakip,Saha,Sema, ikinci_kisim],axis=1)

In [104]:
df['Atilan_Gol']

0       3
1       0
2       6
3       0
4       2
       ..
1487    0
1488    1
1489    1
1490    1
1491    0
Name: Atilan_Gol, Length: 1492, dtype: int64

In [105]:
def y_metot(x):
    
    if x > 0:
        return 1
    else:
        return 0
# df['Atilan_Gol'] = df.apply(lambda x: y_metot(x['Atilan_Gol']), axis=1)

In [106]:
df['Atilan_Gol']

0       3
1       0
2       6
3       0
4       2
       ..
1487    0
1488    1
1489    1
1490    1
1491    0
Name: Atilan_Gol, Length: 1492, dtype: int64

In [107]:
X = X.iloc[:,153:]

In [108]:
# df['Yenilen_Gol'] y1
# df['Atilan_Gol']  y2
y = df['Atilan_Gol']

In [109]:
def compML(X, y, alg):
    #train-test ayrımı
    
    X_train, X_test, y_train, y_test=train_test_split(X, y , test_size=0.20, random_state=42)
    
    #modelleme
    model=alg().fit(X_train,y_train)
    y_pred=model.predict(X_test)
    RMSE=np.sqrt(mean_squared_error(y_test,y_pred))
    model_ismi=alg.__name__ 
    #print(model_ismi,"Modelin İlkel Test Hatası:",RMSE)
    print(model_ismi,"Modelin score yüzdesi:",model.score(X_test,y_test))

In [110]:
dog_olmayan_reg_models=[LGBMRegressor,
       XGBRegressor,
       GradientBoostingRegressor,
       RandomForestRegressor,
       DecisionTreeRegressor,
       MLPRegressor,
       KNeighborsRegressor,
       SVR]

In [111]:
for i in dog_olmayan_reg_models:
    compML(X,y,i)

LGBMRegressor Modelin score yüzdesi: 0.4156631030811856
XGBRegressor Modelin score yüzdesi: 0.275494292911745
GradientBoostingRegressor Modelin score yüzdesi: 0.455694686085762
RandomForestRegressor Modelin score yüzdesi: 0.45419334026223224
DecisionTreeRegressor Modelin score yüzdesi: -0.15110329389190924
MLPRegressor Modelin score yüzdesi: 0.17843852013938377
KNeighborsRegressor Modelin score yüzdesi: 0.097726255196674
SVR Modelin score yüzdesi: 0.21469524796672756


In [73]:
#df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
compML(X,y,LGBMRegressor)

LGBMRegressor Modelin score yüzdesi: 0.4039180177368461


In [74]:
scaler=StandardScaler() 

In [75]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=42)

scaler.fit(X_train) #train veri setinin değişkenleri standartlaştırıldı
X_train_scaled=scaler.transform(X_train) # train veri setinin değişkenleri ölçeklendirildi

scaler.fit(X_test) #test veri setinin değişkenleri standartlaştırıldı
X_test_scaled=scaler.transform(X_test) # test veri setinin değişkenleri ölçeklendirildi

In [51]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt'] 
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10] 
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

In [52]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [39]:
rf_params={"max_depth":[5,8,10],
          "max_features":[2,5,10],
          "n_estimators":[100,500,800],
          "min_samples_split":[2,10,80,100]}

In [53]:
rf_model=RandomForestRegressor(random_state=42).fit(X_train, y_train)

In [72]:
#rf_cv_model=GridSearchCV(rf_model, random_grid,cv=10,n_jobs=-1,verbose=2).fit(X_train,y_train)

In [46]:
rf_cv_model.best_params_

{'max_depth': 10,
 'max_features': 10,
 'min_samples_split': 2,
 'n_estimators': 800}

In [80]:
rf_tuned=RandomForestRegressor(random_state=42,
                               max_depth=10,
                              max_features=10,
                              min_samples_split=2,
                              n_estimators=800).fit(X_train,y_train)

In [81]:
rf_tuned.score(X_test,y_test)

0.39383690341281963

In [82]:
rf_tuned=RandomForestRegressor().fit(X_train,y_train)

In [83]:
rf_tuned.score(X_test,y_test)

0.4612120970330077

In [84]:
y_pred=rf_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

0.982517557436121

In [22]:
gbm_model = LGBMRegressor()

In [23]:
gbm_params={"learning_rate":[0.001,0.1,0.01],
           "max_depth":[3,5,8],
           "n_estimators":[100,200,500],
           "subsample":[1,0.5,0.8],
           "loss":["ls","lad","quantile"]}

In [28]:
gbm_cv_model=GridSearchCV(gbm_model,gbm_params,cv=10,n_jobs=1,verbose=2).fit(X_train,y_train) 

Fitting 10 folds for each of 243 candidates, totalling 2430 fits
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning_rate=0.001, loss=ls, max_depth=3, n_estimators=100, subsample=1; total time=   0.0s
[CV] END learning

In [29]:
gbm_cv_model.best_params_

{'learning_rate': 0.01,
 'loss': 'ls',
 'max_depth': 5,
 'n_estimators': 200,
 'subsample': 1}

In [30]:
gbm_tuned=GradientBoostingRegressor(learning_rate=0.01,
                                   #loss="ls",
                                   max_depth=5,
                                   n_estimators=200,
                                   subsample=1).fit(X_train,y_train)

In [31]:
y_pred=gbm_tuned.predict(X_test)

In [32]:
np.sqrt(mean_squared_error(y_test,y_pred))

1.3368688629914287

In [33]:
gbm_tuned.score(X_test,y_test)

0.13917099029878355